##### *UCSC, Entrez, KEGG, and Reactome*
> **UCSC** genome database is very informative and offers a lot of options that narrow down our search. The genome browser is easy to use and provides good user interface that allows users to look into details by simpily clicking a specific locus on the chromosome.      
**Entrez** allows users to search information in cross databases. It reduces works when users try to compare data in different databases. It allows users to select and sort datas in python, which simplifies data manipulation. However, it stores uncurated data, therefore it could provide undesirable information.      
**KEGG** presents the pathways graphically and also allows users to interact with the graph. It clarifies the interaction among pathways. The pathways are categorized.   
**Reactome** also presents pathways in colorful graphs that makes easier to see. It's easy to zoom in and out.      
I prefer Reactome because of its simplicity, but Entrez seems to be the most complete because it has info in all databases.

In [1]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = 'jinghuawu@berkeley.edu'

In [2]:
orgs = ['Drosophila', 'E.coli', 'human']

glycolysis = ['pyruvate kinase', 'enolase', 
              'Phosphoglycerate mutase', 'phosphofructokinase']

TCA = ['malate dehydrogenase', 'citrate synthase', 
       'aconitase', 'isocitrate dehydrogenase']

pentose_phosphate = ['glucose 6-phosphate dehydrogenase', 'ribose 5-phosphate isomerase', 
                     'transketolase', 'transaldolase']

all_enzymes = glycolysis + TCA + pentose_phosphate

### *Data retrieval*

Variables "organism", "ids", "enzyme", "path", "description" and "ntsql" are lists that store the categorized information, which can be converted to tuples later.

In [3]:
#gene data
organism = []
ids = []
enzyme = []
path = []

for o in orgs: #for each organism
    for e in all_enzymes:
        if e in glycolysis:
            path.append("glycolysis")
        elif e in TCA:
            path.append("TCA")
        elif e in pentose_phosphate:
            path.append("pentose phosphate pathway")
     
        organism.append(o)
        enzyme.append(e)
        handle = Entrez.esearch(db='nucleotide', #can be anything on Entrez (nucleotide, gene, protein, genome)
                                term= o + '[Orgn]' + ' AND ' + e + '[Prot]', #search the term, browser format
                                sort='relevance',
                                idtype='acc', #types of record IDs are returned (acc=accession number) 
                                retmax=1)
        for i in Entrez.read(handle)['IdList']:
            ids.append(i)         

In [4]:
description = []
ntseq = []
for i in ids:			
	handle = Entrez.efetch(db='nucleotide', 
                           id=i, #the first one is the most relevant one
                           rettype='gb', #Retrieval type. 
                           retmode='text')
	record = SeqIO.read(handle, "gb")
	description.append(str(record.description))
	ntseq.append(str(record.seq[:30]))

**Turn data into tuples for later use.**

In [5]:
gene_tuple = []
for n in range(len(ids)):
	gene_tuple.append(tuple([ids[n], description[n], organism[n], enzyme[n], path[n], ntseq[n]]))

#### *Import data to sqlite*

In [6]:
import sqlite3 #provide interface
conn = sqlite3.connect('my.db') #create a Connection object that represents the database. ('example.db')
c = conn.cursor() #create cursor object for method calls later.

**Gene table (36 different genes)**

In [7]:
#gene table
c.execute(
    """CREATE TABLE gene (id INT,
                          name TEXT,
                          description TEXT,
                          organism TEXT,
                          enzyme TEXT,
                          pathway TEXT,
                          ntseq VARCHAR(20));""")

In [8]:
for i in gene_tuple:
    temp = i
    c.execute("""INSERT INTO gene (id, description, organism, enzyme, pathway, ntseq) VALUES (?, ?, ?, ?, ?, ?);""", temp)
conn.commit()

**Pathway table (3 pathways total)**

In [9]:
c.execute("""CREATE TABLE pathway (name TEXT, description TEXT)""")
conn.commit()

In [10]:
c.execute("""INSERT INTO pathway (name, description) VALUES ('glycolysis', 'a metabolic process that occurs during aerobic and anaerobic respiration of living organisms within the cytoplasm.');""")
c.execute("""INSERT INTO pathway (name, description) VALUES ('TCA', 'a metabolic process that occurs during aerobic and anaerobic respiration of living organisms within the cytoplasm.');""")
c.execute("""INSERT INTO pathway (name, description) VALUES ('pentose_phosphate_pathway', 'hexose monophosphate shunt) is a metabolic pathway parallel to glycolysis.');""")
conn.commit()

**enzyme table (12 enzymes total)**

In [11]:
#enzyme table
c.execute("""CREATE TABLE enzyme (name TEXT, function TEXT, enzyme_commission TEXT, path TEXT)""")
conn.commit()

In [12]:
c.execute("""INSERT INTO enzyme (name, function, enzyme_commission, path) 
			 VALUES  ('pyruvate kinase', 'catalyzes the final step of glycolysis', '2.7.1.40', 'glycolysis'),
			 		 ('enolase', 'metalloenzyme responsible for the catalysis of the conversion of 2-phosphoglycerate (2-PG) to phosphoenolpyruvate (PEP)', '4.2.1.11', 'glycolysis'),
			 		 ('Phosphoglycerate mutase', 'any enzyme that catalyzes step 8 of glycolysis', '5.4.2.11', 'glycolysis'),
			 		 ('phosphofructokinase', 'a kinase enzyme that phosphorylates fructose 6-phosphate in glycolysis', '2.7.1.11', 'glycolysis'),
			 		 ('malate dehydrogenase', 'an enzyme that reversibly catalyzes the oxidation of malate to oxaloacetate', '1.1.1.37', 'TCA'),
			 		 ('citrate synthase', 'pace-making enzyme in the first step of the citric acid cycle', '2.3.3.1', 'TCA'), 
			 		 ('aconitase', 'an enzyme that catalyses the stereo-specific isomerization of citrate to isocitrate via cis-aconitate in the tricarboxylic acid cycle', '4.2.1.3', 'TCA'), 
			 		 ('isocitrate dehydrogenase', 'an enzyme that catalyzes the oxidative decarboxylation of isocitrate', '1.1.1.42', 'TCA'),
			 		 ('glucose 6-phosphate dehydrogenase', 'a cytosolic enzyme that catalyzes D-glucose 6-phosphate', '1.1.1.49', 'pentose_phosphate_pathway'),
			 		 ('ribose 5-phosphate isomerase', 'catalyzes the conversion between ribose-5-phosphate (R5P) and ribulose-5-phosphate (Ru5P)', '5.3.1.6', 'pentose_phosphate_pathway'),
                     ('transketolase', 'catalyzes two important reactions, which operate in opposite directions in these two pathways', '2.2.1.1', 'pentose_phosphate_pathway'),
                     ('transaldolase', 'an enzyme (EC 2.2.1.2) of the non-oxidative phase of the pentose phosphate pathway', '2.2.1.2', 'pentose_phosphate_pathway');""")
conn.commit()

In [13]:
c.execute("SELECT * FROM gene;")
print(c.fetchall())

[('XM_023317917.1', None, 'PREDICTED: Drosophila hydei pyruvate kinase (LOC111601379), mRNA', 'Drosophila', 'pyruvate kinase', 'glycolysis', 'TTTCAATACTTAAAAAAAACAAAGTTAATA'), ('XM_023310563.1', None, 'PREDICTED: Drosophila hydei enolase (LOC111596366), mRNA', 'Drosophila', 'enolase', 'glycolysis', 'TTATTTTTGATATATTCAATTCTTAGTTTA'), ('NT_033777.3', None, 'Drosophila melanogaster chromosome 3R', 'Drosophila', 'Phosphoglycerate mutase', 'glycolysis', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'), ('NT_033778.4', None, 'Drosophila melanogaster chromosome 2R', 'Drosophila', 'phosphofructokinase', 'glycolysis', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'), ('NT_033779.5', None, 'Drosophila melanogaster chromosome 2L', 'Drosophila', 'malate dehydrogenase', 'TCA', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'), ('NC_004354.4', None, 'Drosophila melanogaster chromosome X', 'Drosophila', 'citrate synthase', 'TCA', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'), ('NT_033779.5', None, 'Drosophila melanogaster chromosome 2L', 'Drosophila', 'acon

In [14]:
c.execute("SELECT * FROM pathway;")
print(c.fetchall())

[('glycolysis', 'a metabolic process that occurs during aerobic and anaerobic respiration of living organisms within the cytoplasm.'), ('TCA', 'a metabolic process that occurs during aerobic and anaerobic respiration of living organisms within the cytoplasm.'), ('pentose_phosphate_pathway', 'hexose monophosphate shunt) is a metabolic pathway parallel to glycolysis.')]


In [15]:
c.execute("SELECT * FROM enzyme;")
print(c.fetchall())

[('pyruvate kinase', 'catalyzes the final step of glycolysis', '2.7.1.40', 'glycolysis'), ('enolase', 'metalloenzyme responsible for the catalysis of the conversion of 2-phosphoglycerate (2-PG) to phosphoenolpyruvate (PEP)', '4.2.1.11', 'glycolysis'), ('Phosphoglycerate mutase', 'any enzyme that catalyzes step 8 of glycolysis', '5.4.2.11', 'glycolysis'), ('phosphofructokinase', 'a kinase enzyme that phosphorylates fructose 6-phosphate in glycolysis', '2.7.1.11', 'glycolysis'), ('malate dehydrogenase', 'an enzyme that reversibly catalyzes the oxidation of malate to oxaloacetate', '1.1.1.37', 'TCA'), ('citrate synthase', 'pace-making enzyme in the first step of the citric acid cycle', '2.3.3.1', 'TCA'), ('aconitase', 'an enzyme that catalyses the stereo-specific isomerization of citrate to isocitrate via cis-aconitate in the tricarboxylic acid cycle', '4.2.1.3', 'TCA'), ('isocitrate dehydrogenase', 'an enzyme that catalyzes the oxidative decarboxylation of isocitrate', '1.1.1.42', 'TCA')

### *Associative Table*

* gene table    x    pathway table    x    enzyme table *

In [16]:
c.execute("""
CREATE TABLE assoc AS
SELECT gene.id, gene.name AS gene_name, gene.description AS gene_description, gene.organism, gene.ntseq, 
       pathway.name AS pathway, pathway.description AS pathway_description,
       enzyme.name AS enzyme, enzyme.function AS function, enzyme.enzyme_commission AS EC_num
FROM gene, pathway, enzyme
WHERE gene.pathway == pathway.name AND gene.enzyme == enzyme.name AND pathway.name == enzyme.path;""")
conn.commit()

In [17]:
c.execute("SELECT * FROM assoc;")
print(c.fetchall())

[('XM_023317917.1', None, 'PREDICTED: Drosophila hydei pyruvate kinase (LOC111601379), mRNA', 'Drosophila', 'TTTCAATACTTAAAAAAAACAAAGTTAATA', 'glycolysis', 'a metabolic process that occurs during aerobic and anaerobic respiration of living organisms within the cytoplasm.', 'pyruvate kinase', 'catalyzes the final step of glycolysis', '2.7.1.40'), ('XM_023310563.1', None, 'PREDICTED: Drosophila hydei enolase (LOC111596366), mRNA', 'Drosophila', 'TTATTTTTGATATATTCAATTCTTAGTTTA', 'glycolysis', 'a metabolic process that occurs during aerobic and anaerobic respiration of living organisms within the cytoplasm.', 'enolase', 'metalloenzyme responsible for the catalysis of the conversion of 2-phosphoglycerate (2-PG) to phosphoenolpyruvate (PEP)', '4.2.1.11'), ('NT_033777.3', None, 'Drosophila melanogaster chromosome 3R', 'Drosophila', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNN', 'glycolysis', 'a metabolic process that occurs during aerobic and anaerobic respiration of living organisms within the cytoplasm.

>#### Comments:     
1. To avoid crashing the website, I only limited the sequence to 30 nucleotides here.   
2. As one of the disadvantages of Entrez, the search results from Entrez are inconsistent, therefore some sequences (especially those in e.coli) do not contain useful information.

> **1. There are relationships between enzymes and pathways—some enzymes belong to certain pathways. Do any belong to multiple pathways? Is this a one-to-many or many-to-many relationship?**       
To my knowledge, the enzymes I found here belong to only one pathway, therefore it's a one-to-one relationship. But if there are, it will be a many-to-many relationship because some enzymes might belong to many pathways and a pathway might contain enzymes that are also in its parental table.

> **2. There is an order to enzymes within pathways. How can the order be represented in a table?**     
Assigning enzymes a name or index, then sort the table by names or index.    

> **3. Genes in the gene table encode enzymes in the enzyme table. How can this be represented? Is this a one-to-one, one-to-many, or many-to-many relationship, and in which direction?**    
This can be represented by adding a columns that indicates the pathway(s) the enzyme involved to the end of the gene table, and use boolean statement to filter the rows. Gene to enzyme is a one-to-one relationship because one gene corresponds to one enzyme.


***Some manual modifications on the table***

In [18]:
c.execute("""UPDATE assoc SET gene_name = "AC02" WHERE id == 'AH007467.3';""")
c.execute("""UPDATE assoc SET gene_name = "G6pd" WHERE id == 'AY364534.1';""")
c.execute("""UPDATE assoc SET gene_name = "PFKM" WHERE id == 'AH002936.2';""")
c.execute("""UPDATE assoc SET gene_name = "PHGDH" WHERE id == 'NM_006623.3';""")
c.execute("""UPDATE assoc SET gene_name = "IDH2" WHERE id == 'KU639670.1';""")
c.execute("""UPDATE assoc SET gene_name = "TALDO1" WHERE id == 'NM_006755.1';""")
conn.commit()

> **Comment**:
Due to the inconsistentcy of Entrez search results, not all given sequences are mRNA sequence, thus not all have gene_name. 


# !! Don't run the following code !!

In [ ]:
#Delete table
c.execute("""DROP TABLE gene;""")
c.execute("""DROP TABLE enzyme;""")
c.execute("""DROP TABLE pathway;""")
c.execute("""DROP TABLE assoc;""")
conn.commit()